In [201]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
%matplotlib inline

Creacion de los sets de tests y features. Voy a probar primero con todo junto

In [202]:
df_palabras = pd.read_csv('../features/palabras_clave.csv')
df_word2vec = pd.read_csv('../features/word2vec.csv')
df_features = df_palabras.join(df_word2vec,how = 'inner')

df_train = pd.read_csv('../dataset/train.csv', index_col='id')
df_test = pd.read_csv('../dataset/test.csv', index_col='id')

In [203]:
features_cols = df_features.columns.tolist()
df_train = df_train.join(df_features, how='inner')

X = df_train[features_cols]
Y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=17)

In [204]:
param_grid = { 
    'n_estimators': [x for x in range(100, 500, 100)],
    'max_depth': [3, 5, 10],
    'colsample_bytree': [0.5, 0.7, 0.9]
}

In [175]:
xgb_classifier = xgb.XGBClassifier(random_state=17, learning_rate=0.05)
CV_xgb = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='f1', n_jobs=4)
CV_xgb.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.05, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=17, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'colsample_bytree': [0.5, 0.7, 0.9],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300,

In [176]:
CV_xgb.best_params_

{'colsample_bytree': 0.9, 'max_depth': 10, 'n_estimators': 400}

In [177]:
def try_classifier(classifier):
    predictions = classifier.predict(X_test)
    predictions_with_prob = classifier.predict_proba(X_test)[:,1]
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(f'Verdaderos Negativos: {tn}')
    print(f'Falsos Negativos: {fn}')
    print(f'Verdaderos Positivos: {tp}')
    print(f'Falsos Positivos: {fp}')
    print()
    print(f'f1 score: {f1_score(y_test,  predictions)}')

In [178]:
try_classifier(CV_xgb)

Verdaderos Negativos: 621
Falsos Negativos: 458
Verdaderos Positivos: 209
Falsos Positivos: 235

f1 score: 0.37623762376237624


In [179]:
df_submit = df_test.join(df_features, how='inner')[features_cols]
df_submit.shape

(3263, 10)

In [180]:
kaggle_pred = CV_xgb.predict(df_submit)

In [181]:
df_submit['target'] = kaggle_pred

In [182]:
df_submit.reset_index(inplace = True)
df_submit.rename(columns={'index':'id'}, inplace=True)
df_submit =df_submit[['id','target']]

In [183]:
df_submit[['id', 'target']].to_csv('../submits/submit_word2vec.csv',index=False)

In [184]:
df =  pd.read_csv('../submits/submit_word2vec.csv')
df.columns

Index(['id', 'target'], dtype='object')

Cambiemos el modelo a RF.

In [200]:
df_train

,keyword,location,text,target,mencion_divina,contiene_risa,contiene_hora_especifica,word_count,similitud_debris,similitud_derailment,similitud_wreckage,similitud_outbreak,similitud_typhoon,similitud_bombing
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,False,False,False,7,0.713841,0.714017,0.713508,0.712389,0.714055,0.713592
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False,False,False,10,0.799382,0.799647,0.799015,0.797832,0.799691,0.799226
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,False,False,False,15,0.865596,0.865885,0.865229,0.863757,0.865917,0.865369
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,False,False,False,12,0.915847,0.916139,0.915416,0.914009,0.916160,0.915573
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,False,False,False,6,0.995076,0.995555,0.994548,0.993035,0.995633,0.994971
8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,False,False,False,7,0.999372,0.999718,0.998901,0.997451,0.999782,0.999137
10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,False,False,False,6,0.999463,0.999790,0.998977,0.997471,0.999843,0.999286
13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,False,False,False,6,0.994596,0.995245,0.994145,0.993058,0.995246,0.994828
14,NaN,NaN,There's an emergency evacuation happening now ...,1,False,False,False,9,0.997757,0.998193,0.997290,0.995591,0.998165,0.997605
15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,False,False,False,2,0.999494,0.999795,0.998990,0.997612,0.999847,0.999210


In [186]:
full_model = df_train
model = full_model[full_model['target'].notnull()]
labels = model['target']
model = model.drop('target', axis=1)
model = model.drop(['text','location','keyword'],axis=1)
model

,mencion_divina,contiene_risa,contiene_hora_especifica,word_count,similitud_debris,similitud_derailment,similitud_wreckage,similitud_outbreak,similitud_typhoon,similitud_bombing
1,False,False,False,7,0.713841,0.714017,0.713508,0.712389,0.714055,0.713592
4,False,False,False,10,0.799382,0.799647,0.799015,0.797832,0.799691,0.799226
5,False,False,False,15,0.865596,0.865885,0.865229,0.863757,0.865917,0.865369
6,False,False,False,12,0.915847,0.916139,0.915416,0.914009,0.916160,0.915573
7,False,False,False,6,0.995076,0.995555,0.994548,0.993035,0.995633,0.994971
8,False,False,False,7,0.999372,0.999718,0.998901,0.997451,0.999782,0.999137
10,False,False,False,6,0.999463,0.999790,0.998977,0.997471,0.999843,0.999286
13,False,False,False,6,0.994596,0.995245,0.994145,0.993058,0.995246,0.994828
14,False,False,False,9,0.997757,0.998193,0.997290,0.995591,0.998165,0.997605
15,False,False,False,2,0.999494,0.999795,0.998990,0.997612,0.999847,0.999210


In [226]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
features_cols = df_features.columns.tolist()

X = df_train[features_cols]
Y = df_train['target']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=17)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  8.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth':

In [227]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 2,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 200}

In [229]:
clf1 = RandomForestClassifier(bootstrap= True,
 max_depth= 100,
 max_features = 2,
 min_samples_leaf= 5,
 min_samples_split= 12,
 n_estimators= 200)

In [231]:
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=100, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=12,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [241]:
result = clf1.predict_proba(X_test)

In [242]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [244]:

from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.5142349969874876